In [1]:
import geopandas as gpd
import pandas as pd
from _bus_cost_utils import GCS_PATH
from calitp_data_analysis.sql import to_snakecase
from IPython.display import HTML, Markdown, display

pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [2]:
all_data = pd.read_parquet(
    f"{GCS_PATH}cleaned_no_outliers_cpb_analysis_data_merge.parquet"
)

In [3]:
# raw FTA
fy24 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/FY2024_Bus_Awards_/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryPolygon&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="
fy23 = "https://services.arcgis.com/xOi1kZaI0eWDREZv/ArcGIS/rest/services/2023_06_12_Awards/FeatureServer/0/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&resultType=none&distance=0.0&units=esriSRUnit_Meter&relationParam=&returnGeodetic=false&outFields=*&returnGeometry=true&featureEncoding=esriDefault&multipatchOption=xyFootprint&maxAllowableOffset=&geometryPrecision=&outSR=&defaultSR=&datumTransformation=&applyVCSProjection=false&returnIdsOnly=false&returnUniqueIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&returnQueryGeometry=false&returnDistinctValues=false&cacheHint=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&having=&resultOffset=&resultRecordCount=&returnZ=false&returnM=false&returnTrueCurves=false&returnExceededLimitFeatures=true&quantizationParameters=&sqlFormat=none&f=pgeojson&token="

fta_fy_24 = gpd.read_file(fy24)
fta_fy_23 = gpd.read_file(fy23)

In [4]:
# raw TIRCP
tircp_file_name = "raw_TIRCP Tracking Sheets 2_1-10-2024.xlsx"
tircp_name = "Project Tracking"

# read in data
tircp_data = pd.read_excel(f"{GCS_PATH}{tircp_file_name}", sheet_name=tircp_name)

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [5]:
# raw DGS
# params
file_17c = "raw_17c compiled-Proterra Compiled Contract Usage Report .xlsx"
file_17b = "raw_17b compiled.xlsx"
sheet_17c = "Proterra "
sheet_17b = "Usage Report Template"

# read in data
dgs_17c = pd.read_excel(f"{GCS_PATH}{file_17c}", sheet_name=sheet_17c)
dgs_17b = pd.read_excel(f"{GCS_PATH}{file_17b}", sheet_name=sheet_17b)

# add new column to identify source
dgs_17c["source"] = "17c"
dgs_17b["source"] = "17b"

In [6]:
raw_data_dict = {
    "dgs2": dgs_17c,
    "dgs1": dgs_17b,
    "tircp": tircp_data,
    "fta24": fta_fy_24,
    "fta23": fta_fy_23,
}

raw_data_list = [
    dgs_17c,
    dgs_17b,
    tircp_data,
    fta_fy_24,
    fta_fy_23,
]

In [7]:
for name, df in raw_data_dict.items():
    display(name, list(df.columns))

'dgs2'

['Supplier Contract Usage ID',
 'Ordering Agency Name',
 'State (S) or Local (L) agency',
 'Purchasing Authority Number                    (for State departments)',
 'Agency Billing Code',
 'Purchase Order Number',
 'Purchase Order Date',
 'Delivery Date',
 'Contract Line Item Number (CLIN)                (RFP ID)',
 'UNSPSC Code\n(Version 10)',
 'Manufacturer Part Number (OEM #)',
 'Manufacturer (OEM)',
 'SKU # / Item #',
 'Item Description',
 'Unit of Measure',
 'Quantity in \nUnit of Measure\n',
 'EPP (Y/N)',
 'Quantity',
 'List Price/MSRP',
 'Index Date / Catalog Version',
 'Contract Unit Price',
 'Contract Discount',
 'Extended Contract Price Paid',
 'Core/ NonCore',
 'Group ID/ Segment ID',
 'source']

'dgs1'

['Supplier Contract Usage ID',
 'Ordering Agency Name',
 'State (S) or Local (L) agency',
 'Purchasing Authority Number                    (for State departments)',
 'Agency Billing Code',
 'Purchase Order Number',
 'Purchase Order Date',
 'Delivery Date',
 'Contract Line Item Number (CLIN)                (RFP ID)',
 'UNSPSC Code\n(Version 10)',
 'Manufacturer Part Number (OEM #)',
 'Manufacturer (OEM)',
 'SKU # / Item #',
 'Item Description',
 'Unit of Measure',
 'Quantity in \nUnit of Measure\n',
 'EPP (Y/N)',
 'Quantity',
 'List Price/MSRP',
 'Index Date / Catalog Version',
 'Contract Unit Price',
 'Contract Discount',
 'Extended Contract Price Paid',
 'Total with options per unit',
 'Grand Total',
 'source']

'tircp'

['Award Year',
 'Project #',
 'Grant Recipient',
 'Project Title',
 'PPNO',
 'District',
 'County',
 'Project Description',
 'bus_count',
 'Master Agreement Number',
 'Master Agreement Expiration Date',
 'Project Manager',
 'Regional Coordinator',
 'Technical Assistance-CALITP (Y/N)',
 'Technical Assistance-Fleet (Y/N)',
 'Technical Assistance-Network Integration (Y/N)',
 'Technical Assistance-Priority Population (Y/N)',
 'Total Project Cost',
 'TIRCP Award Amount ($)',
 'Allocated Amount',
 'Unallocated Amount',
 'Percentage Allocated',
 'Expended Amount',
 'Other Funds Involved',
 'Award Cycle',
 'Is SB1?',
 'Is GGRF?',
 'Is IIJA?',
 'ON SHS?',
 'CalITP',
 'Estimated TIRCP GHG Reductions',
 'Estemated Project Completion',
 'Estimated TIRCP GHG Reductions2',
 'Increased Ridership',
 'Service Integration',
 'Improve Safety',
 'Project Readiness',
 'Funding Leverage',
 'Multi-Agency Coordination/Integration',
 'AB 1550 Community Benefits',
 'Housing Co-Benefits',
 'Local Agency Address'

'fta24'

['Application_Number_',
 'Agency_Name',
 'Low_No_BBF',
 'Project_Description',
 'Number_of_Buses_',
 'Funding_Amount',
 'Project_Title',
 'Project_Type',
 'Display_Type',
 'Overall_Propulsion_Type',
 'Specific_Propulsion_Type',
 'Areas_Served',
 'Congressional_Districts',
 'Address_Line_1',
 'City',
 'State',
 'Additional_Zip',
 'Zip_Code',
 'X',
 'Y',
 'FID2',
 'geometry']

'fta23'

['Application_Number',
 'FTA_Region',
 'Project_Sponsor',
 'Project_Title',
 'Project_Description',
 'Original_Statement_of_Work',
 'Funding',
 'Number_of_Buses',
 'Traditional_Buses',
 'Low_Emission_Buses',
 'Zero_Emission_Buses',
 'Project_Type',
 'Display_Type',
 'Areas_Served',
 'Congressional_Districts',
 'Propulsion_Type',
 'Address1',
 'Address2',
 'City',
 'State',
 'Zip',
 'X',
 'Y',
 'ObjectId',
 'geometry']

## RE: feedback from REBEL
see `refactor_notes_2.md` for specifics

# Specific points to tackle

## "We would likely recommend explicitly separating vehicles out by size/type in all analysis to increase accuracy (and provide important data points for agencies for whom the smaller transit vehicles are crucial)."
- explore the bus size values again, can i get more granular?
- where am i getting the bus size/type from? usually the `project description` column or similar

In [ ]:
# fta
display(
    list(fta_fy_24.columns),  # Project_Description, Project_Title
    list(fta_fy_23.columns),  # Project_Description, Project_Title
)

In [31]:
vehicle_type = [
    "Articulated bus",
    "Automated guideway vehicle",
    "Automobile",
    "Over-the-road bus",
    "Bus",
    "Cable car",
    "Cutaway",
    "Double decker bus",
    "Ferryboat",
    "Heavy Rail passenger car",
    "Inclined plane vehicle",
    "Light Rail vehicle",
    "Minivan",
    "Commuter Rail locomotive",
    "Commuter Rail passenger coach",
    "Commuter Rail, self-propelled passenger car",
    "School bus",
    "Sports utility vehicle (SUV)",
    "Trolleybus",
    "Aerial tramway vehicle",
    "Van",
    "Vintage trolley/Streetcar",
]

test_pattern = "|".join(vehicle_type)

In [ ]:
fta_fy_24[["Project_Description"]][fta_fy_24["Project_Description"].str.contains("bus")]

# The FTA data does not have specific bus type data (cut away, 30ft, 40ft, etc). 
# Mainly just says if buses were purchase or not. Does not go into detail what type or kind of bus was purchased

In [45]:
# tircp
tircp_data.columns

Index(['Award Year', 'Project #', 'Grant Recipient', 'Project Title', 'PPNO',
       'District', 'County', 'Project Description', 'bus_count',
       'Master Agreement Number', 'Master Agreement Expiration Date',
       'Project Manager', 'Regional Coordinator',
       'Technical Assistance-CALITP (Y/N)', 'Technical Assistance-Fleet (Y/N)',
       'Technical Assistance-Network Integration (Y/N)',
       'Technical Assistance-Priority Population (Y/N)', 'Total Project Cost',
       'TIRCP Award Amount ($)', 'Allocated Amount', 'Unallocated Amount',
       'Percentage Allocated', 'Expended Amount', 'Other Funds Involved',
       'Award Cycle', 'Is SB1?', 'Is GGRF?', 'Is IIJA?', 'ON SHS?', 'CalITP',
       'Estimated TIRCP GHG Reductions', 'Estemated Project Completion',
       'Estimated TIRCP GHG Reductions2', 'Increased Ridership',
       'Service Integration', 'Improve Safety', 'Project Readiness',
       'Funding Leverage', 'Multi-Agency Coordination/Integration',
       'AB 1550 Com

In [68]:
tircp_data["bus_count"].isna().value_counts()

True     89
False    35
Name: bus_count, dtype: int64

In [ ]:
tircp_data[["Project Description","bus_count"]][tircp_data["bus_count"]>0]

# there are some size/type data in tircp, but not much.

In [76]:
# dgs
display(dgs_17c.columns,
list(dgs_17b.columns))

Index(['Supplier Contract Usage ID', 'Ordering Agency Name',
       'State (S) or Local (L) agency',
       'Purchasing Authority Number                    (for State departments)',
       'Agency Billing Code', 'Purchase Order Number', 'Purchase Order Date',
       'Delivery Date',
       'Contract Line Item Number (CLIN)                (RFP ID)',
       'UNSPSC Code\n(Version 10)', 'Manufacturer Part Number (OEM #)',
       'Manufacturer (OEM)', 'SKU # / Item #', 'Item Description',
       'Unit of Measure', 'Quantity in \nUnit of Measure\n', 'EPP (Y/N)',
       'Quantity', 'List Price/MSRP', 'Index Date / Catalog Version',
       'Contract Unit Price', 'Contract Discount',
       'Extended Contract Price Paid', 'Core/ NonCore', 'Group ID/ Segment ID',
       'source'],
      dtype='object')

['Supplier Contract Usage ID',
 'Ordering Agency Name',
 'State (S) or Local (L) agency',
 'Purchasing Authority Number                    (for State departments)',
 'Agency Billing Code',
 'Purchase Order Number',
 'Purchase Order Date',
 'Delivery Date',
 'Contract Line Item Number (CLIN)                (RFP ID)',
 'UNSPSC Code\n(Version 10)',
 'Manufacturer Part Number (OEM #)',
 'Manufacturer (OEM)',
 'SKU # / Item #',
 'Item Description',
 'Unit of Measure',
 'Quantity in \nUnit of Measure\n',
 'EPP (Y/N)',
 'Quantity',
 'List Price/MSRP',
 'Index Date / Catalog Version',
 'Contract Unit Price',
 'Contract Discount',
 'Extended Contract Price Paid',
 'Total with options per unit',
 'Grand Total',
 'source']

In [75]:
# dgs
display(dgs_17c["Manufacturer (OEM)"].value_counts(),
dgs_17b["Manufacturer (OEM)"].value_counts()
)

Proterra, Inc.    29
Name: Manufacturer (OEM), dtype: int64

New Flyer of America Inc    33
Name: Manufacturer (OEM), dtype: int64

In [77]:
display(dgs_17c["Item Description"].value_counts(),
dgs_17b["Item Description"].value_counts()
)

# DGS does have some bus size/type data, 
# BUT the source is a report of all the Proterra and New Flyer buses

Additional 220 kWH battery pack           13
Options - Configurables                   13
35 Foot Low Floor Battery Electric Bus     9
Other - Tools                              8
40 Foot Low Floor Battery Electric Bus     7
Options - Charging Equipment               5
Options - Configurables (DuoPower)         3
Other - Training, manuals and Tools        3
Other - Spare Parts                        3
Options - Warranty                         2
Extended Warranty                          2
Options - Configurables (non taxable)      2
Other - Warranty                           2
Options - Configurables (ProDrive)         1
Other - Training, manuals                  1
Name: Item Description, dtype: int64

40 foot battery electric bus                                                     5
40 Foot Low Floor Battery Electric Bus                                           4
40 Foot Low Floor Hydrogen Electic Bus                                           4
35 foot battery electric bus                                                     3
Extended warranty for Propulsion System and Battery                              3
40 foot battery electric                                                         2
Extended warranty for Propulsion System, Battery, HVAC, APC and Camera system    1
60 foot battery electric Bus                                                     1
Diagnostic equipment and tooling                                                 1
Extended warranty for Air Compressor                                             1
40 Foot Low Floor Fuel Cell Electric Bus                                         1
Training, Manuals, Diagnostic equipment and tooling                              1
Manu

## "Providing time series data to be able to see >progression in prices over time"
- explore the data sources for any date/time fields
- when were the buses ordered and/or received.
- the TIRCP/DGS may have this data, FTA will not

In [78]:
# tircp
tircp_data.sample(2)

#can get award year, 

,Award Year,Project #,Grant Recipient,Project Title,PPNO,District,County,Project Description,bus_count,Master Agreement Number,Master Agreement Expiration Date,Project Manager,Regional Coordinator,Technical Assistance-CALITP (Y/N),Technical Assistance-Fleet (Y/N),Technical Assistance-Network Integration (Y/N),Technical Assistance-Priority Population (Y/N),Total Project Cost,TIRCP Award Amount ($),Allocated Amount,Unallocated Amount,Percentage Allocated,Expended Amount,Other Funds Involved,Award Cycle,Is SB1?,Is GGRF?,Is IIJA?,ON SHS?,CalITP,Estimated TIRCP GHG Reductions,Estemated Project Completion,Estimated TIRCP GHG Reductions2,Increased Ridership,Service Integration,Improve Safety,Project Readiness,Funding Leverage,Multi-Agency Coordination/Integration,AB 1550 Community Benefits,Housing Co-Benefits,Local Agency Address,Local Agency City,Local Agency Zip,Local Agency Contact,Local Agency Email,Local Agency Phone Number,Comments/Additional Contacts,S California
22,2016,9,Peninsula Corridor Joint Powers Board (PCJPB),Peninsula Corridor Electrification Project,CP022,4,VAR,Electrification of approximately 51 miles of existing Caltrain Corridor and procurement of 96 EMUs.,NaN,64PCJPBMA01,2026-06-30 00:00:00,Jason Finley,Shannon Simonds,No,No,No,No,1980400000,387000000,386750000,250000,0.999354,1.773165e+07,Yes,2,N,Y,N,NaN,NaN,"734,000 MTCO2",NaN,High,High,High,High,Medium,High,Medium-High,Medium,NaN,"1250 San Carlos Ave\nSan Carlos, 94070",San Carlos,94070,"Peter Skinner, Manager of Grants and Fund Programming",skinnerp@samtrans.com,(650) 622-7818,NaN,No
54,2018,27,Southern California Regional Rail Authority,Southern California Optimized Rail Expansion (SCORE),CP033,7,VAR,"Delivers more frequent, more reliable rail services throughout Southern California, with station reconfiguration with run-though tracks for Metrolink and Pacific Surfliner trains at Los Angeles Union Station to improve train movement through the station, and 30-min services on multiple Metrolink corridors in the LA Basin. Includes significant investments to improve the frequency and performance of rail services to Moorpark, Santa Clarita, San Bernardino, Riverside, and Orange County. Part of a high-performance long-range vision.",NaN,64SCRRAMA A02,2031-08-02 00:00:00,Noey Rios,Anthony Serna,No,No,Yes,No,2049700000,875708000,407632000,468076000,0.465488,1.170575e+08,NaN,3,Y,Y,N,NaN,NaN,"5,714,000 MTCO2e",NaN,High,High,High,High,Medium,Medium-High,High,Medium-High,NaN,"900 Wilshire Blvd, Ste 1500\nLos Angeles, CA 90017",Los Angeles,90017,"Tom Schamber, Interim Director, Grants",SchamberT@scrra.net,(213) 494-7289,NaN,Yes


In [81]:
# dgs
dgs_17c.sample(1)

# can use purchase order date,


,Supplier Contract Usage ID,Ordering Agency Name,State (S) or Local (L) agency,Purchasing Authority Number (for State departments),Agency Billing Code,Purchase Order Number,Purchase Order Date,Delivery Date,Contract Line Item Number (CLIN) (RFP ID),UNSPSC Code\n(Version 10),Manufacturer Part Number (OEM #),Manufacturer (OEM),SKU # / Item #,Item Description,Unit of Measure,Quantity in \nUnit of Measure\n,EPP (Y/N),Quantity,List Price/MSRP,Index Date / Catalog Version,Contract Unit Price,Contract Discount,Extended Contract Price Paid,Core/ NonCore,Group ID/ Segment ID,source
56,102865447,Napa Valley Transportation Authority,L,NaN,NaN,21-2002,2021-08-26,Q2 2022,NaN,NaN,NaN,NaN,NaN,Options - Charging Equipment,Ea,1,Y,1,NaN,NaN,141700.0,NaN,141700.0,Core,2,17c


In [82]:
dgs_17b.sample(1)

,Supplier Contract Usage ID,Ordering Agency Name,State (S) or Local (L) agency,Purchasing Authority Number (for State departments),Agency Billing Code,Purchase Order Number,Purchase Order Date,Delivery Date,Contract Line Item Number (CLIN) (RFP ID),UNSPSC Code\n(Version 10),Manufacturer Part Number (OEM #),Manufacturer (OEM),SKU # / Item #,Item Description,Unit of Measure,Quantity in \nUnit of Measure\n,EPP (Y/N),Quantity,List Price/MSRP,Index Date / Catalog Version,Contract Unit Price,Contract Discount,Extended Contract Price Paid,Total with options per unit,Grand Total,source
20,154186,San Diego Metro,L,NaN,NaN,4500040166,2021-06-24,July-23,9.3,NaN,XE60,New Flyer of America Inc,SR-2664,60 foot battery electric Bus,EA,1,NaN,12,NaN,NaN,1218483.94,Volume Discount Applied,14621807.28,1563298.63,18759583.56,17b


## "Data by manufacture"
- check the TIRCP and DGS data for this

In [12]:
# tircp

tircp_data,

In [13]:
# dgs
dgs_17c,
dgs_17b,

## "Data by purchasing vehicle/contract"
- make this more clear i TIRCP and DGS data

In [ ]:
dgs_17c,
            dgs_17b,
            tircp_data,
            fta_fy_24,
            fta_fy_23